# Cài đặt thư viện


- Cài đặt thư viện Selenium, Chromium
- Thiết lập cấu hình Ubuntu cho Chromium

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF
# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg
# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500
Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300
Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.pfj3DCgdm8/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.rzlbODsGND/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Uvokmm6n1h/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://archive.ubuntu.com/ubuntu 

# Thu thập dữ liệu thời tiết

Thiết lập Webdriver cho Chrome

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import re

service = Service(executable_path=r'/usr/bin/chromedriver')
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=options)

Thu thập dữ liệu thành phố HCM từ 1/1/2021 -> 31/5/2023

In [3]:
from urllib.parse import urlparse
from selenium.common.exceptions import NoSuchElementException

list_web = [
  {'name':'HoChiMinh','url':'https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/{}-{}'},
]

# Tạo DataFrame trống
data = pd.DataFrame()

for i in list_web:
  city_name = i['name']
  url_crawl = i['url']

  for year in range(2023, 2024):
    start_month = 6
    end_month = 6

    for month in range(start_month, end_month + 1):
      url = url_crawl.format(year, month)

      # Truy cập trang web và cào dữ liệu
      driver.get(url)
      print(f"Đang thu thập dữ liệu từ trang web: {url}")

      # Chờ đến khi có dữ liệu table
      while True:
        try:
          table = driver.find_element(By.XPATH, "//table[@aria-labelledby='History days']")
          break
        except NoSuchElementException:
          time.sleep(1)

      # Chờ đến khi có dữ liệu column
      while True:
        try:
          columns = table.find_elements(By.XPATH, ".//table[@aria-labelledby='Days data']")
          if columns:
              break
        except NoSuchElementException:
          time.sleep(1)

      # Lấy giá trị thời gian và địa điểm
      url_parts = url.split('/')
      date_value = url_parts[-1]

      time_col = columns[0]
      time_rows = time_col.find_elements(By.XPATH, ".//td")
      time_values = [f"{date_value}-{time.text}" for time in time_rows][1:]

      def dailyweather(column):
        position = [1, 2, 0]
        values = []
        for i in position:
          cells = column.find_elements(By.XPATH, f".//td[position() mod 3 = {i}]")
          values.append([cell.text for cell in cells][1:])
        return values

      # Lấy dữ liệu nhiệt độ
      temperature = columns[1]
      t_max_values, t_avg_values, t_min_values = dailyweather(temperature)

      # Điểm sương
      dewpoint = columns[2]
      d_max_values, d_avg_values, d_min_values = dailyweather(dewpoint)

      # Độ ẩm
      humidity = columns[3]
      h_max_values, h_avg_values, h_min_values = dailyweather(humidity)

      # Tốc độ gió
      windspeed = columns[4]
      w_max_values, w_avg_values, w_min_values = dailyweather(windspeed)

      # Tạo DataFrame từ dữ liệu thu thập được trong tháng
      month_data = pd.DataFrame({
        'Time': time_values,
        'Location': city_name,
        'MaxTemperature': t_max_values,
        'AvgTemperature': t_avg_values,
        'MinTemperature': t_min_values,
        'MaxDewPoint': d_max_values,
        'AvgDewPoint': d_avg_values,
        'MinDewPoint': d_min_values,
        'MaxHumidity': h_max_values,
        'AvgHumidity': h_avg_values,
        'MinHumidity': h_min_values,
        'MaxWindspeed': w_max_values,
        'AvgWindspeed': w_avg_values,
        'MinWindspeed': w_min_values
      })

      # Gộp dữ liệu của tháng hiện tại vào DataFrame tổng
      data = pd.concat([data, month_data], ignore_index=True)

# Ghi DataFrame vào tệp CSV
data.to_csv('data.csv', index=False)
print("Thu thập dữ liệu thành công")

Đang thu thập dữ liệu từ trang web: https://www.wunderground.com/history/monthly/vn/ho-chi-minh-city/VVTS/date/2023-6
Thu thập dữ liệu thành công


In [4]:
from IPython.display import display
import pandas as pd
data_filter = pd.read_csv('./data.csv')
display(data_filter.head(5))

,Time,Location,MaxTemperature,AvgTemperature,MinTemperature,MaxDewPoint,AvgDewPoint,MinDewPoint,MaxHumidity,AvgHumidity,MinHumidity,MaxWindspeed,AvgWindspeed,MinWindspeed
0,2023-6-1,HoChiMinh,93,86.2,81,82,81.2,79,100,85.2,66,17,11.4,7
1,2023-6-2,HoChiMinh,93,86.5,82,84,81.2,79,100,84.1,66,16,10.2,3
2,2023-6-3,HoChiMinh,95,87.0,82,84,81.0,77,100,82.8,59,16,10.1,7
3,2023-6-4,HoChiMinh,95,85.1,75,84,80.1,75,100,85.4,63,25,8.3,1
4,2023-6-5,HoChiMinh,91,81.9,77,82,79.2,75,100,92.1,71,15,5.7,0


In [5]:
data_filter.isnull().sum()

Time              0
Location          0
MaxTemperature    0
AvgTemperature    0
MinTemperature    0
MaxDewPoint       0
AvgDewPoint       0
MinDewPoint       0
MaxHumidity       0
AvgHumidity       0
MinHumidity       0
MaxWindspeed      0
AvgWindspeed      0
MinWindspeed      0
dtype: int64

In [6]:
# Kiểm tra có dòng dữ liệu nào có giá trị bằng 0 trong các cột
columns_check= ['MaxTemperature','AvgTemperature','MinTemperature',
                'MaxDewPoint','AvgDewPoint','MinDewPoint',
                'MaxHumidity','AvgHumidity','MinHumidity',
                'MaxWindspeed','AvgWindspeed']

zero_rows = data_filter[(data_filter[columns_check] == 0).any(axis=1)]
sum_zero = len(zero_rows)

# Lưu vị trí
zero_location = zero_rows.index

# In ra các dòng dữ liệu có giá trị bằng 0
display(zero_rows)
print("Tổng số dòng có dữ liệu bằng 0: ",sum_zero)

,Time,Location,MaxTemperature,AvgTemperature,MinTemperature,MaxDewPoint,AvgDewPoint,MinDewPoint,MaxHumidity,AvgHumidity,MinHumidity,MaxWindspeed,AvgWindspeed,MinWindspeed


Tổng số dòng có dữ liệu bằng 0:  0


In [7]:
data_convert = data_filter.copy()
columns = ['MaxTemperature', 'AvgTemperature', 'MinTemperature','MaxDewPoint', 'AvgDewPoint', 'MinDewPoint']

for i in columns:
    data_convert[i] = round((data_filter[i] - 32) / 1.8, 1 if 'Avg' in i else 0)

display(data_convert)

,Time,Location,MaxTemperature,AvgTemperature,MinTemperature,MaxDewPoint,AvgDewPoint,MinDewPoint,MaxHumidity,AvgHumidity,MinHumidity,MaxWindspeed,AvgWindspeed,MinWindspeed
0,2023-6-1,HoChiMinh,34.0,30.1,27.0,28.0,27.3,26.0,100,85.2,66,17,11.4,7
1,2023-6-2,HoChiMinh,34.0,30.3,28.0,29.0,27.3,26.0,100,84.1,66,16,10.2,3
2,2023-6-3,HoChiMinh,35.0,30.6,28.0,29.0,27.2,25.0,100,82.8,59,16,10.1,7
3,2023-6-4,HoChiMinh,35.0,29.5,24.0,29.0,26.7,24.0,100,85.4,63,25,8.3,1
4,2023-6-5,HoChiMinh,33.0,27.7,25.0,28.0,26.2,24.0,100,92.1,71,15,5.7,0
5,2023-6-6,HoChiMinh,32.0,27.9,26.0,27.0,26.1,25.0,100,90.5,70,21,6.7,0
6,2023-6-7,HoChiMinh,32.0,28.6,26.0,27.0,26.4,25.0,100,87.6,70,14,8.0,0
7,2023-6-8,HoChiMinh,31.0,28.8,26.0,27.0,26.1,25.0,100,85.5,70,15,8.5,3
8,2023-6-9,HoChiMinh,32.0,28.5,26.0,28.0,26.6,25.0,100,89.2,70,15,7.8,1
9,2023-6-10,HoChiMinh,32.0,28.2,26.0,28.0,26.5,25.0,100,90.9,70,20,9.4,3


In [8]:
data_convert.to_csv("data_prefuture.csv", index=False)
print("Dữ liệu xử lý đã được lưu vào file data_prefuture.csv")

Dữ liệu xử lý đã được lưu vào file data_prefuture.csv
